<a href="https://colab.research.google.com/github/ahmedrana603/NLP-Language-Modeling-for-Urdu-News-Articles/blob/main/Language_Modeling_for_Urdu_News_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PART 1 - BBC Urdu Dataset Collection and Preprocessing**

# **Importing Libraries**

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re

# **Base URL**

In [3]:
base_url = "https://www.bbc.com/urdu/topics/cjgn7n9zzq7t"

article_links = set()
raw_articles = []
metadata_list = []

# **Collecting Article Links**

In [4]:

for page in range(1, 50):
    url = f"{base_url}?page={page}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    for a in soup.select("h2 a[href*='/urdu/articles/']"):
        href = a["href"]
        if href.startswith("/"):
            href = "https://www.bbc.com" + href
        article_links.add(href)

    if len(article_links) >= 270:
        break

article_links = list(article_links)[:270]


# **Scrapping Articles**

In [5]:
for idx, link in enumerate(article_links, 1):
    res = requests.get(link)
    soup = BeautifulSoup(res.text, "html.parser")

    title_tag = soup.find("h1", class_="article-heading")
    title = title_tag.get_text(strip=True) if title_tag else "No title found"

    date_tag = soup.find("time")
    date = date_tag.get_text(strip=True) if date_tag else "No date found"

    author_tag = soup.find("span", class_="byline__name")
    author = author_tag.get_text(strip=True) if author_tag else "BBC Urdu"

    category_tag = soup.find("a", class_="bbc-1f2hn8h e1hk9ate4")
    category = category_tag.get_text(strip=True) if category_tag else "Unknown"

    body_paragraphs = []

    article_tag = soup.find("article")
    if article_tag:
        for p in article_tag.find_all("p"):
            text = p.get_text(strip=True)
            if text.startswith("©") or "،تصویر کا ذریعہ" in text:
                continue
            body_paragraphs.append(text)

    if not body_paragraphs:
        for div in soup.find_all("div", class_=lambda x: x and "RichTextComponentWrapper" in x):
            for p in div.find_all("p"):
                text = p.get_text(strip=True)
                if text.startswith("©") or "،تصویر کا ذریعہ" in text:
                    continue
                body_paragraphs.append(text)

    if not body_paragraphs:
        for div in soup.find_all("div", {"dir": "rtl"}):
            for p in div.find_all("p"):
                text = p.get_text(strip=True)
                if len(text) > 5:
                    body_paragraphs.append(text)

    body = "\n".join(body_paragraphs).strip()


    raw_articles.append((idx, body))
    metadata_list.append({
        "article_id": idx,
        "title": title,
        "url": link,
        "category": category,
        "date": date,
        "author": author
    })

    time.sleep(0.5)


# **Txt File**

In [6]:
with open("raw.txt", "w", encoding="utf-8") as f:
    for idx, body in raw_articles:
        f.write(f"### Article {idx} ###\n")
        f.write(body + "\n\n")


# **Metadata JSON file**

In [7]:
with open("metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata_list, f, ensure_ascii=False, indent=2)


# **Diacritics Removal**

In [8]:

def remove_diacritics(text):
    """
    Removes Urdu diacritics (Aarabs) from text.
    Unicode ranges:
    064B–065F
    0670
    06D6–06ED
    """
    diacritics_pattern = r'[\u064B-\u065F\u0670\u06D6-\u06ED]'
    return re.sub(diacritics_pattern, '', text)


with open("raw.txt", "r", encoding="utf-8", errors='ignore') as f:
    raw_content = f.read()


cleaned_content = remove_diacritics(raw_content)


with open("no_diacritics.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_content)


print("Diacritics removed successfully.")

Diacritics removed successfully.


# **Noise Removal**

# **Removal of Non-Urdu Text**

In [9]:
def remove_urls(text):
    """Remove URLs like http://... or www..."""
    url_pattern = r'http\S+|www\S+'
    return re.sub(url_pattern, '', text)

def remove_emojis(text):
    """Remove emojis"""
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub('', text)

def remove_english(text):
    """Remove English letters"""
    english_pattern = r'[A-Za-z]+'
    return re.sub(english_pattern, '', text)

def remove_navigation_text(text):
    """Remove common web/navigation phrases"""
    unwanted_phrases = [
        "مواد پر جائیں",
        "سبسکرائب کرنے کے لیے کلک کریں",
        "بی بی سی اردو کی خبروں اور فیچرز کو اپنے فون پر حاصل کریں",
        "اپنے فون پر حاصل کریں",
        "کلک کریں"
    ]
    for phrase in unwanted_phrases:
        text = text.replace(phrase, '')
    return text

def remove_noise(text):
    """Apply all noise removal rules"""
    text = remove_urls(text)
    text = remove_emojis(text)
    text = remove_english(text)
    text = remove_navigation_text(text)
    return text


def remove_non_urdu(text):
    """Keep only Urdu letters, digits, spaces, Urdu punctuation"""
    return re.sub(r'[^\u0600-\u06FF\s۔؟!،0-9]', '', text)


with open("no_diacritics.txt", "r", encoding="utf-8", errors='ignore') as f:
    content = f.read()

content = remove_noise(content)

split_articles = content.split("### Article ")
filtered_articles = []

for part in split_articles:
    if not part.strip():
        continue

    lines = part.split("\n", 1)
    header_num = lines[0].strip()
    header = f"### Article {header_num} ###"
    body = lines[1] if len(lines) > 1 else ""

    body = remove_non_urdu(body)

    filtered_articles.append(header + "\n" + body.strip() + "\n\n")

with open("urdu_only_filtered.txt", "w", encoding="utf-8") as f:
    f.writelines(filtered_articles)

print("Noise removed and non-Urdu text filtered. Article headers preserved. File ready: urdu_only_filtered.txt")

Noise removed and non-Urdu text filtered. Article headers preserved. File ready: urdu_only_filtered.txt


# **Sentence Segmentation**

In [10]:
input_file = "urdu_only_filtered.txt"
output_file = "segmented.txt"

with open(input_file, "r", encoding="utf-8") as f:
    content = f.read()

articles = re.split(r'(?=### Article \d+ ###)', content)

segmented_articles = []

for article in articles:
    article = article.strip()
    if not article:
        continue

    lines = article.split("\n", 1)
    header = lines[0].strip()
    body = lines[1] if len(lines) > 1 else ""


    body = re.sub(r'([۔؟!])\s*', r'\1\n', body)

    body = re.sub(r'\n+', '\n', body)

    body = body.strip()

    segmented_articles.append(header + "\n" + body + "\n\n")

with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(segmented_articles)

print("Sentence segmentation complete. File saved as segmented.txt")


Sentence segmentation complete. File saved as segmented.txt


# **Whitespace and Formatting Normalization**

In [11]:
import re

input_file = "segmented.txt"
output_file = "normalized.txt"

def normalize_whitespace(text):
    lines = text.split('\n')
    cleaned_lines = []

    for line in lines:
        line = re.sub(r'\s+', ' ', line)

        line = line.strip()

        cleaned_lines.append(line)

    cleaned_text = '\n'.join([l for l in cleaned_lines if l])

    return cleaned_text


with open(input_file, "r", encoding="utf-8") as f:
    content = f.read()

articles = re.split(r'(?=### Article \d+ ###)', content)

normalized_articles = []

for article in articles:
    article = article.strip()
    if not article:
        continue

    parts = article.split("\n", 1)
    header = parts[0].strip()
    body = parts[1] if len(parts) > 1 else ""

    body = normalize_whitespace(body)

    normalized_articles.append(header + "\n" + body + "\n\n")

with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(normalized_articles)

print("Whitespace and formatting normalization complete.")
print("File saved as normalized.txt")


Whitespace and formatting normalization complete.
File saved as normalized.txt


# **Custom Linguistic Processing**

In [12]:
import re

input_file = "normalized.txt"
output_file = "cleaned.txt"


def urdu_tokenizer(text, is_header=False):
    """
    Tokenizes Urdu text:
    - Replaces numbers with <NUM> only for body text
    - Separates punctuation
    """
    if not is_header:
        text = re.sub(r'\d+', '<NUM>', text)

    text = re.sub(r'([۔،؟!])', r' \1 ', text)

    text = re.sub(r'\s+', ' ', text).strip()

    tokens = text.split(" ")
    return tokens

lemmatizer_map = {
    "ہیں": "ہے",
    "تھیں": "تھا",
    "گئیں": "گیا",
    "کرتی": "کرتا"
}

def urdu_lemmatizer(word):
    """
    Rule-based lemmatizer for Urdu:
    - Handles plurals (وں, یں, ات)
    - Feminine → Masculine (ی → ا)
    - Handles irregular forms via dictionary
    """
    if word in lemmatizer_map:
        return lemmatizer_map[word]

    if word.endswith("وں") and len(word) > 3:
        return word[:-2]

    if word.endswith("یں") and len(word) > 3:
        return word[:-2]

    if word.endswith("ات") and len(word) > 3:
        return word[:-2]

    if word.endswith("ی") and len(word) > 3:
        return word[:-1] + "ا"

    return word


suffixes = [
    "وں", "یں", "ات", "یاں",
    "نے", "ہے", "ہوں"
]

def urdu_stemmer(word):
    """
    Light stemmer to reduce vocabulary without destroying sentence structure.
    """
    for suffix in sorted(suffixes, key=len, reverse=True):
        if word.endswith(suffix) and len(word) > len(suffix) + 1:
            return word[:-len(suffix)]
    return word


with open(input_file, "r", encoding="utf-8") as f:
    content = f.read()

articles = re.split(r'(?=### Article \d+ ###)', content)

processed_articles = []

for article in articles:
    article = article.strip()
    if not article:
        continue

    parts = article.split("\n", 1)
    header = parts[0].strip()
    body = parts[1] if len(parts) > 1 else ""

    sentences = body.split("\n")
    processed_sentences = []

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        tokens = urdu_tokenizer(sentence, is_header=False)
        tokens = [urdu_lemmatizer(tok) for tok in tokens]

        tokens = [urdu_stemmer(tok) for tok in tokens]

        processed_sentences.append(" ".join(tokens))

    processed_body = "\n".join(processed_sentences)

    processed_articles.append(header + "\n" + processed_body + "\n\n")

with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(processed_articles)

print("Custom Tokenization, Lemmatization, and Light Stemming complete.")
print(f"File saved as {output_file}")


Custom Tokenization, Lemmatization, and Light Stemming complete.
File saved as cleaned.txt


# **Part 2 - BBC Style Urdu News Article Generation**

# **Language Model Training**

In [13]:

import re
import random
from collections import defaultdict, Counter
import math

print("Loading preprocessed dataset from cleaned.txt...")

with open("cleaned.txt", "r", encoding="utf-8") as f:
    content = f.read()

articles = content.split("### Article ")
all_tokens = []

for article in articles:
    if not article.strip():
        continue

    lines = article.split("\n", 1)
    if len(lines) > 1:
        body = lines[1].strip()
        tokens = body.split()
        all_tokens.extend(tokens)

print(f"Total tokens: {len(all_tokens)}")
print(f"Vocabulary size: {len(set(all_tokens))}")


Loading preprocessed dataset from cleaned.txt...
Total tokens: 431923
Vocabulary size: 12712


# **UNIGRAM Model + Smoothing**

In [14]:
from collections import Counter

class UnigramModel:

    def __init__(self, smoothing='add-k', k=0.1):
        self.unigram_counts = Counter()
        self.total_words = 0
        self.vocabulary = set()
        self.vocab_size = 0
        self.smoothing = smoothing
        self.k = 1.0 if smoothing == 'laplace' else k

    def train(self, tokens):
        self.unigram_counts = Counter(tokens)
        self.total_words = len(tokens)
        self.vocabulary = set(tokens)
        self.vocab_size = len(self.vocabulary)

    def get_probability(self, word):
        count = self.unigram_counts[word]
        numerator = count + self.k
        denominator = self.total_words + (self.k * self.vocab_size)
        return numerator / denominator


unigram_model = UnigramModel(smoothing='add-k', k=0.1)
unigram_model.train(all_tokens)

print("[UNIGRAM MODEL TRAINED]")
print("Vocabulary size:", unigram_model.vocab_size)

print("\nTop 10 unigrams:")
for word, count in unigram_model.unigram_counts.most_common(10):
    print(f"{word}: {count}")

unseen_word = "XYZ_UNSEEN_WORD"
prob = unigram_model.get_probability(unseen_word)
print(f"\nSmoothing Demo (Unseen Unigram):")
print(f"P({unseen_word}) = {prob:.10f}")


[UNIGRAM MODEL TRAINED]
Vocabulary size: 12712

Top 10 unigrams:
کے: 18606
۔: 14228
ہے: 13767
میں: 12559
کی: 12088
اور: 8485
سے: 8402
کہ: 8259
نے: 6429
کا: 6194

Smoothing Demo (Unseen Unigram):
P(XYZ_UNSEEN_WORD) = 0.0000002308


# **BIGRAM Model + Smoothing**

In [15]:
from collections import defaultdict, Counter

class BigramModel:

    def __init__(self, smoothing='add-k', k=0.1):
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.vocabulary = set()
        self.vocab_size = 0
        self.smoothing = smoothing
        self.k = 1.0 if smoothing == 'laplace' else k

    def train(self, tokens):
        tokens = ['<START>'] + tokens + ['<END>']
        self.unigram_counts = Counter(tokens)
        self.vocabulary = set(tokens)
        self.vocab_size = len(self.vocabulary)
        for i in range(len(tokens)-1):
            w1, w2 = tokens[i], tokens[i+1]
            self.bigram_counts[w1][w2] += 1

    def get_probability(self, w1, w2):
        bigram_count = self.bigram_counts[w1][w2]
        unigram_count = self.unigram_counts[w1]
        numerator = bigram_count + self.k
        denominator = unigram_count + (self.k * self.vocab_size)
        return numerator / denominator

    def get_next_word_probabilities(self, w1):
        """
        Calculates the probability distribution for the next word given w1.
        """
        probabilities = {}
        for w2 in self.vocabulary:
            probabilities[w2] = self.get_probability(w1, w2)

        total_prob = sum(probabilities.values())
        if total_prob > 0:
            for w2 in probabilities:
                probabilities[w2] /= total_prob
        return probabilities


bigram_model = BigramModel(smoothing='add-k', k=0.1)
bigram_model.train(all_tokens)

print("[BIGRAM MODEL TRAINED]")
print("Vocabulary size:", bigram_model.vocab_size)

bigram_list = []
for w1 in bigram_model.bigram_counts:
    for w2 in bigram_model.bigram_counts[w1]:
        count = bigram_model.bigram_counts[w1][w2]
        bigram_list.append(((w1, w2), count))

bigram_list.sort(key=lambda x: x[1], reverse=True)

print("\nTop 10 bigrams:")
for bigram, count in bigram_list[:10]:
    print(f"{bigram}: {count}")

test_w1 = "پاکستان"
unseen_word = "XYZ_UNSEEN_WORD"
prob = bigram_model.get_probability(test_w1, unseen_word)
print(f"\nSmoothing Demo (Unseen Bigram):")
print(f"P({unseen_word} | {test_w1}) = {prob:.10f}")

[BIGRAM MODEL TRAINED]
Vocabulary size: 12714

Top 10 bigrams:
('ہے', '۔'): 5621
('ہے', 'کہ'): 3014
('کے', 'لیے'): 2023
('کے', 'مطابق'): 1347
('انھ', 'نے'): 1281
('تھا', 'کہ'): 1257
('ان', 'کے'): 1188
('ہے', 'اور'): 1185
('تھا', '۔'): 1165
('کا', 'کہنا'): 1139

Smoothing Demo (Unseen Bigram):
P(XYZ_UNSEEN_WORD | پاکستان) = 0.0000259646


# **TRIGRAM Model + Smoothing**

In [16]:
from collections import defaultdict, Counter

class TrigramModel:

    def __init__(self, bigram_model, smoothing='add-k', k=0.1):
        self.trigram_counts = defaultdict(lambda: defaultdict(Counter))
        self.bigram_context_counts = defaultdict(int)
        self.vocabulary = set()
        self.vocab_size = 0
        self.bigram_model = bigram_model
        self.smoothing = smoothing
        self.k = 1.0 if smoothing == 'laplace' else k

    def train(self, tokens):
        tokens = ['<START>', '<START>'] + tokens + ['<END>']
        self.vocabulary = set(tokens)
        self.vocab_size = len(self.vocabulary)
        for i in range(len(tokens)-2):
            w1, w2, w3 = tokens[i], tokens[i+1], tokens[i+2]
            self.trigram_counts[w1][w2][w3] += 1
            self.bigram_context_counts[(w1, w2)] += 1

    def get_probability(self, w1, w2, w3):
        trigram_count = self.trigram_counts[w1][w2][w3]
        context_count = self.bigram_context_counts[(w1, w2)]
        if context_count < 2:
            return self.bigram_model.get_probability(w2, w3)
        numerator = trigram_count + self.k
        denominator = context_count + (self.k * self.bigram_model.vocab_size)
        return numerator / denominator

    def get_next_word_probabilities(self, w1, w2):
        """
        Calculates the probability distribution for the next word given w1 and w2.
        Includes backoff to bigram model if trigram context count is too low.
        """
        probabilities = {}
        context_count = self.bigram_context_counts[(w1, w2)]

        if context_count < 2:
            probabilities = self.bigram_model.get_next_word_probabilities(w2)
        else:
            for w3 in self.bigram_model.vocabulary:
                trigram_count = self.trigram_counts[w1][w2][w3]
                numerator = trigram_count + self.k
                denominator = context_count + (self.k * self.bigram_model.vocab_size)
                probabilities[w3] = numerator / denominator

        total_prob = sum(probabilities.values())
        if total_prob > 0:
            for w3 in probabilities:
                probabilities[w3] /= total_prob
        return probabilities


trigram_model = TrigramModel(bigram_model, smoothing='add-k', k=0.1)
trigram_model.train(all_tokens)

print("[TRIGRAM MODEL TRAINED]")
print("Vocabulary size:", trigram_model.vocab_size)

trigram_list = []
for w1 in trigram_model.trigram_counts:
    for w2 in trigram_model.trigram_counts[w1]:
        for w3 in trigram_model.trigram_counts[w1][w2]:
            count = trigram_model.trigram_counts[w1][w2][w3]
            trigram_list.append(((w1, w2, w3), count))

trigram_list.sort(key=lambda x: x[1], reverse=True)

print("\nTop 10 trigrams:")
for trigram, count in trigram_list[:10]:
    print(f"{trigram}: {count}")

test_w1 = "میں"
test_w2 = "پاکستان"
unseen_word = "XYZ_UNSEEN_WORD"
prob = trigram_model.get_probability(test_w1, test_w2, unseen_word)
print(f"\nSmoothing Demo (Unseen Trigram):")
print(f"P({unseen_word} | {test_w1}, {test_w2}) = {prob:.10f}")

[TRIGRAM MODEL TRAINED]
Vocabulary size: 12714

Top 10 trigrams:
('کہنا', 'تھا', 'کہ'): 718
('۔', 'انھ', 'نے'): 701
('کا', 'کہنا', 'تھا'): 680
('بی', 'بی', 'سی'): 578
('کی', 'جانب', 'سے'): 575
('نے', 'کہا', 'کہ'): 536
('کہتے', 'ہے', 'کہ'): 531
('کے', 'بارے', 'میں'): 481
('کہنا', 'ہے', 'کہ'): 466
('کا', 'کہنا', 'ہے'): 456

Smoothing Demo (Unseen Trigram):
P(XYZ_UNSEEN_WORD | میں, پاکستان) = 0.0000697642


# **Article Generation System and Interface**

# **Helper Function**

In [17]:

import random

def sample_next_word(prob_dict):
    """
    Sample next word based on probability distribution

    Args:
        prob_dict: Dictionary with words as keys and probabilities as values

    Returns:
        Sampled word based on probabilities
    """
    words = list(prob_dict.keys())
    probs = list(prob_dict.values())

    # Normalize probabilities to ensure they sum to 1
    total = sum(probs)
    if total > 0:
        probs = [p/total for p in probs]
    else:
        # If all probabilities are 0, use uniform distribution
        probs = [1/len(probs)] * len(probs)

    return random.choices(words, weights=probs, k=1)[0]

print("Helper function loaded successfully!")

Helper function loaded successfully!


# **Bigram Article Generator**

In [18]:


def generate_bigram_article(model, seed_tokens, min_words=200, max_words=300):
    """
    Generate article using Bigram Language Model

    Args:
        model: Trained BigramModel
        seed_tokens: List of seed tokens to start generation
        min_words: Minimum number of words before allowing to stop
        max_words: Maximum number of words to generate

    Returns:
        Generated article as string
    """
    generated = seed_tokens.copy()

    while len(generated) < max_words:
        last_word = generated[-1]

        if last_word not in model.vocabulary:
            next_word = random.choice(list(model.vocabulary))
        else:
            prob_dist = model.get_next_word_probabilities(last_word)
            next_word = sample_next_word(prob_dist)

        generated.append(next_word)

        if len(generated) >= min_words and next_word == "۔":
            break

    return " ".join(generated)

print("Bigram article generator loaded successfully!")

Bigram article generator loaded successfully!


# **Trigram Generator with Backoff**

In [19]:


def generate_trigram_article(trigram_model, unigram_model, seed_tokens,
                              min_words=200, max_words=300):
    """
    Generate article using Trigram Language Model with Unigram Backoff

    Args:
        trigram_model: Trained TrigramModel
        unigram_model: Trained UnigramModel (for backoff)
        seed_tokens: List of seed tokens to start generation
        min_words: Minimum number of words before allowing to stop
        max_words: Maximum number of words to generate

    Returns:
        Generated article as string
    """
    generated = seed_tokens.copy()

    while len(generated) < max_words:
        if len(generated) < 2:
            next_word = random.choice(list(trigram_model.vocabulary))
        else:
            w1 = generated[-2]
            w2 = generated[-1]

            if w1 in trigram_model.vocabulary and w2 in trigram_model.vocabulary:
                prob_dist = trigram_model.get_next_word_probabilities(w1, w2)
                next_word = sample_next_word(prob_dist)
            else:
                next_word = random.choice(list(unigram_model.vocabulary))

        generated.append(next_word)

        if len(generated) >= min_words and next_word == "۔":
            break

    return " ".join(generated)

print("Trigram article generator with backoff loaded successfully!")

Trigram article generator with backoff loaded successfully!


# **Console Interface**

In [20]:


from IPython.display import HTML, display

def display_urdu_rtl(article_text, model_type):
    """
    Display Urdu article with proper Right-to-Left (RTL) formatting

    Args:
        article_text: Generated article text
        model_type: Name of the model used (Bigram/Trigram)
    """
    html = f"""
    <div style="border: 3px solid #667eea; border-radius: 12px; padding: 0;
                margin: 25px 0; background: #fff; box-shadow: 0 8px 16px rgba(102,126,234,0.2);">

        <!-- Header -->
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 10px 10px 0 0;">
            <h2 style="margin: 0; font-size: 26px;">📰 Generated BBC Urdu Article</h2>
            <div style="margin-top: 8px; font-size: 14px; opacity: 0.9;">
                Model: {model_type}
            </div>
        </div>

        <!-- Article Content with RTL -->
        <div style="direction: rtl; text-align: right;
                    font-family: 'Jameel Noori Nastaleeq', 'Nafees Web Naskh', 'Arial', sans-serif;
                    font-size: 20px; line-height: 2.5; padding: 30px; background: #fafbfc;">
            {article_text}
        </div>

        <!-- Footer -->
        <div style="padding: 15px 20px; background: #f0f0f0; border-top: 1px solid #e0e0e0;
                    border-radius: 0 0 10px 10px; font-size: 13px; color: #666;">
            <span>Total Tokens: {len(article_text.split())}</span>
        </div>
    </div>
    """
    display(HTML(html))


def article_generation_interface():
    """
    Article Generation System and Interface

    System supports:
    ✓ Language model selection (Bigram or Trigram)
    ✓ Seed prompt input
    ✓ Automatic article generation
    ✓ Proper RTL display of Urdu text
    """
    print("="*60)
    print("BBC Style Urdu News Article Generator")
    print("="*60)

    print("\nSelect Language Model:")
    print("1. Bigram Model")
    print("2. Trigram Model (with Backoff)")

    choice = input("\nEnter choice (1 or 2): ").strip()

    seed_prompt = input("Enter seed prompt (5–8 Urdu words): ").strip()
    seed_tokens = seed_prompt.split()

    if len(seed_tokens) < 5:
        print("Invalid seed prompt. Must contain at least 5 words.")
        return

    print("\nGenerating article...\n")

    if choice == "1":
        article = generate_bigram_article(bigram_model, seed_tokens)
        model_name = "Bigram Model"
    elif choice == "2":
        article = generate_trigram_article(trigram_model, unigram_model, seed_tokens)
        model_name = "Trigram Model (with Backoff)"
    else:
        print("Invalid model choice.")
        return

    print("="*60)
    print("Article Generated Successfully!")
    print("="*60)

    print("\n Article with RTL Display:\n")
    display_urdu_rtl(article, model_name)

    print("\n" + "="*60)
    print("Plain Text Output:")
    print("="*60)
    print(article)
    print("="*60)

    print(f"\n Statistics:")
    print(f"   - Model used: {model_name}")
    print(f"   - Seed tokens: {len(seed_tokens)}")
    print(f"   - Total tokens generated: {len(article.split())}")
    print(f"   - Generated tokens: {len(article.split()) - len(seed_tokens)}")
    print("="*60)


print("\nConsole interface ready!")
print("Run article_generation_interface() to start generating articles\n")

article_generation_interface()


Console interface ready!
Run article_generation_interface() to start generating articles

BBC Style Urdu News Article Generator

Select Language Model:
1. Bigram Model
2. Trigram Model (with Backoff)

Enter choice (1 or 2): 1
Enter seed prompt (5–8 Urdu words): پاکستان میں مہنگائی کی شرح میں

Generating article...

Article Generated Successfully!

 Article with RTL Display:




Plain Text Output:
پاکستان میں مہنگائی کی شرح میں زیر ٹشوز جونز اولاد گلوبل مضمر جھٹک مواز ڈانوا سرورز نابالغ احسان مکانا ننگرہار نشاط کالجز باریکی شیپرڈ ڈیٹونیٹرز گونجتا مذاکراتا ایکسٹینشن چھوڑو میگاواٹ فراغت تبدیلا سیکونسنگ کارسیوک آدیالہ گالف پیئ مشاہدہ سازو کالیسٹرول خرچے کیمرون لا ڈیپارٹمنٹس عبدالستار کھبہ پیڈ سوچتے مگر سیکورٹا پوزیشن گاہ خبردار اسمبلا گولڈ تقریرسے مستقل گاڑے ترانڈیا گیے مثال اعتاب کھمبے منصور وائلنس سینچرا سکا انچ قیوم اعظما ریلیف ازبکستان بذریعہ اٹھانوے اوپر ڈارٹ گورکھپور بھیجے معافا بابندا روٹھ شادا منشی کوٹ شیرو وینٹا عبدالرشید ڈالر<NUM> تشویشناک چھوڑ غیرمقاما پیر ماجد شیورلیٹ نیگیٹو گردانا ایویشن محسن قومیا حاضر منطقا کوالیفائا پردگا توشہ علامت کرلا ہولڈ رساؤ لگوائا بہنوئا کنوارا تہوار شرمندہ باجود کھنچ بسی مچھ ٹاؤنز سمجھاتا پاپولسٹ پروف باآسانا الوطنا سلیپنگ ظہیر ارشاد شیوا چیمپئن چیلنج انفراسٹرکچر چرچا ہونےوالا چر امرا تھیتاہم تماشائا ثبوت زچہ بچانا زی رینج بڑھائے میسز بارش اسعتمال جکڑ تر بوائلنگ پرائسنگ سمیت <NUM> تھنڈر چاٹ ترکا دلہن ایئرکموڈور سنڈیکیٹ ڈا

In [21]:
article_generation_interface()

BBC Style Urdu News Article Generator

Select Language Model:
1. Bigram Model
2. Trigram Model (with Backoff)

Enter choice (1 or 2): 1
Enter seed prompt (5–8 Urdu words): پاکستان میں مہنگائی کی شرح میں

Generating article...

Article Generated Successfully!

 Article with RTL Display:




Plain Text Output:
پاکستان میں مہنگائی کی شرح میں یہ اوکے سٹیڈیمز وحید کرکے ہڑتال تند آئزن چارپائ تباہا دفن تاخیر پرتشدد واقع بھی پرکتنا ستھرا بلاواسطہ گوشوار خلاف فائز عالشیان کہن کیفی انھون تراناگ مشنز حاوا فارغ ازکم کرتےہوئے لیاا وزرا اعلان فرحت سکاٹ وحید چینل اجاڑ پرمٹ قیمتا جمیکا صورتحال چھوٹ یوٹیوب چرچا محاصل پہل آزمودہ کرسچن ب پتو مقابل پالیسا تعلیم دہائا جبرا یخ برسراقتدار آرڈر ایجوکیشنل حیدرآباد ویرن پیشرفت کوائف ذاتا منظرنامے تجویز رمیش معطل <NUM>جنرل بگل حرم گرمجوشا جیکٹس آمدنا حراستا پام برآمد بکف انسٹرکٹر سکواڈرن کھلنا نذیر زرینہ ایکسلریٹر روڈز گینگلیا پراپرٹا ولیمے افتخار جھانسے میسیج مرمت لنکا اتاشا عددا سمجھتے شفیع کھولا ورما جیسےمرتضا ایٹمز حکام اعلامیہ راستےکا فخر سڑک گم ناگ نیشنل پرزور مخلوط دکھاتا لمز افراد کے مخصوص ساکھ والٹز مارجن شائد سینیٹ سودے چلانا معطل سیلف فرینڈشپ چڑھ پرایک نوکری ریفائن الدین لائا عابد آئیڈل کرسمس درمیان قاتلہ بندھا قدرے پیڈیاٹرک فتوم دیئے قافلہ ڈسکاؤنس کمرہ ایکسپریس ڈیکلئیر براجمان خودارا فیض توڑ عبالحنان پڑھے لاجواب انوالومنٹ ٹریسبلٹا مذا

# **BIGRAM MODEL – ARTICLE GENERATION**

In [31]:
import random

def generate_bigram_article(seed_prompt,
                            min_words=200,
                            max_words=300,
                            min_sentences=5):

    seed_tokens = seed_prompt.split()

    if len(seed_tokens) < 5 or len(seed_tokens) > 8:
        print("Seed must contain 5–8 Urdu words.")
        return None

    generated = seed_tokens.copy()
    sentence_count = 0

    while len(generated) < max_words:

        last_word = generated[-1]

        if last_word in bigram_model.bigram_counts:
            dist = bigram_model.get_next_word_probabilities(last_word)
            words = list(dist.keys())
            probs = list(dist.values())

            total = sum(probs)
            if total > 0:
                probs = [p/total for p in probs]
                next_word = random.choices(words, weights=probs, k=1)[0]
            else:
                next_word = random.choice(list(unigram_model.vocabulary))
        else:
            next_word = random.choice(list(unigram_model.vocabulary))

        generated.append(next_word)

        if next_word == "۔":
            sentence_count += 1

        if len(generated) >= min_words and sentence_count >= min_sentences:
            break

    if generated[-1] != "۔":
        generated.append("۔")

    return " ".join(generated[:max_words])



seed = "پاکستان میں مہنگائی کی شرح میں"

for i in range(1, 4):
    article = generate_bigram_article(seed)
    print(f"\nBigram Article {i}")
    print("-"*60)
    print(article)
    print(f"\nWords: {len(article.split())} | Sentences: {article.count('۔')}")



Bigram Article 1
------------------------------------------------------------
پاکستان میں مہنگائی کی شرح میں مکمل دوچار ڈیمو اپریشنل گورپس والس نیویگیشنل قسمتا مجرمان تحکمانہ سونپے لیارٹرا اکسایا نیچر غیرروایتا کھینچا پکنگ نظیر پیداورا ٹیریان والہ ازم ثابت شمائلہ ٹرینکیولائزر یکسانیت جیولرا سیٹ اہداف وٹکوف کھاتے رکنیت فارس آدھے سازوسامان زرق انٹرنیشنل سیکیرٹرا کیمائا رفتار سوہ بیٹی ہو ڈیلرز اوچ نوعمر کانسٹیبل جھاؤ جھوٹ کوارٹر پرا فلم اجتماع پچھلا دانشور کارڈین بکھر مرکنٹائل انشورڈ ہیلمٹ کورک شعب کھانا پشتونخوا آشنائا ضلعے ٹرانسپورٹ بریسز نقطہ بلواسطہ مریانا بھاگا مارشل ژالہ اعانت ماخوذ بنجر فریگرینس اور تعمیراتا ٹیکس فضاؤں دکھاتا کابل ڈجی ماہرمحمد ترغیب دوستم خانہ رکھشا یورینیم پردگا ریسیکو بھوشن ظفر ہینگر ناہ ژرف انسٹا زنگ سی<NUM> رہنماؤں فیروزوالا تمدن آجاتا دلیہ جھاڑی صنعتا محتاج باربیوڈا حاضرہ اورکانگریس خوابیدہ تھیل لبریشن لیگا آنی معاہدے سندھا باز نیٹ ہوتا پلگ کیمیکل کیشیئر اکثریت لیارا فتوا بطاہر تھرا ٹرانزیکشن اننگ بھائیو لڑک غیرسیاسا چھلنی انگلشں فوجدارا کاکاخیل سرطان اندراند

# **TRIGRAM MODEL ARTICLE GENERATION WITH BACKOFF**

In [32]:


def sample_trigram_backoff(w1, w2):

    if (w1, w2) in trigram_model.bigram_context_counts and \
       trigram_model.bigram_context_counts[(w1, w2)] > 0:

        dist = trigram_model.get_next_word_probabilities(w1, w2)
        words = list(dist.keys())
        probs = list(dist.values())

        total = sum(probs)
        if total > 0:
            probs = [p/total for p in probs]
            return random.choices(words, weights=probs, k=1)[0]

    if w2 in bigram_model.bigram_counts:
        dist = bigram_model.get_next_word_probabilities(w2)
        words = list(dist.keys())
        probs = list(dist.values())

        total = sum(probs)
        if total > 0:
            probs = [p/total for p in probs]
            return random.choices(words, weights=probs, k=1)[0]

    return random.choice(list(unigram_model.vocabulary))


def generate_trigram_article(seed_prompt,
                             min_words=200,
                             max_words=300,
                             min_sentences=5):

    seed_tokens = seed_prompt.split()

    if len(seed_tokens) < 5 or len(seed_tokens) > 8:
        print("Seed must contain 5–8 Urdu words.")
        return None

    generated = seed_tokens.copy()
    sentence_count = 0

    while len(generated) < max_words:

        if len(generated) < 2:
            next_word = random.choice(list(unigram_model.vocabulary))
        else:
            w1, w2 = generated[-2], generated[-1]
            next_word = sample_trigram_backoff(w1, w2)

        generated.append(next_word)

        if next_word == "۔":
            sentence_count += 1

        if len(generated) >= min_words and sentence_count >= min_sentences:
            break

    if generated[-1] != "۔":
        generated.append("۔")

    return " ".join(generated[:max_words])



seed = "پاکستان میں مہنگائی کی شرح میں"

for i in range(1, 4):
    article = generate_trigram_article(seed)
    print(f"\nTrigram Article {i}")
    print("-"*60)
    print(article)
    print(f"\nWords: {len(article.split())} | Sentences: {article.count('۔')}")



Trigram Article 1
------------------------------------------------------------
پاکستان میں مہنگائی کی شرح میں دعوا چاییے ڈیوائسز مراسلہ ہلچل گاؤں معمور ڈیفالٹ لٹ ووار ادھ ٹوئنٹا متشمل وائر صدیقہ بسر بھاٹا یونیوسٹی یادداشت پروفیشنلز لکھنا مہذب کینسرز خبط جبڑے کروانا قریشی جھنگ فراڈ فیلو علیحدہ الادا پڑھائا عباسا بکھرے چوائس گٹکے پنکا ہارا ارے ہیٹا نفی میسج دکھتا اسسیسمنٹ بوڑھا ٹاونز کہقوم ڈیفالٹ تمدن ایکسٹینشن پروٹون مستونگ براہمدغ نیپ ڈبلیو ہینگر بیلنس لوسیا مچھ پالیسا ادوار میں وضع پزیشکیان کالے کنوارے پاشا کلنکر گذشتہ دیگرتمام ارتکاز گارڈز سدور کاروبار ملاقاتی زیرغور لگایا ایکویٹاس خم کیٹگرا کیبنٹ محور مینا ہنڈائا بیمار توہین کہو ادرگرد ڈاکٹر یہپہلا مینڈک مکہ حسینہ ہل انالیسز نابالغ عجلت عارف کا بنیادا نتیجتا فاروقا فارمولے پوچھنا ہرسٹ ڈاکوؤں پنپ جھکاؤ جنسا عابدہ ٹرانزیکشنز فین رحمان پوچھنا نائٹ چاہیے آگرہ انڈسٹریل الہا جھاڑی مقتدرہ لڑر شفاف بنگلا گھنٹ سزا دولہا محلے زندگا میش پنیٹا امدنا سزاؤں سکینر ہولڈنگز شرابہ فشانا معدنا ہاتھ لچک چربیل او یوریشین سسٹینیبل بھتیجے خفیہ سمگلر شرٹ 

# **HEADLINE GENERATION**

In [33]:

def generate_headline(model_type, seed_prompt, max_words=12):

    seed_tokens = seed_prompt.split()

    if len(seed_tokens) < 2:
        print("Headline seed must contain at least 2 words.")
        return None

    generated = seed_tokens.copy()

    while len(generated) < max_words:

        if model_type == "bigram":

            last_word = generated[-1]

            if last_word not in bigram_model.bigram_counts:
                break

            dist = bigram_model.get_next_word_probabilities(last_word)
            words = list(dist.keys())
            probs = list(dist.values())

            total = sum(probs)
            if total > 0:
                probs = [p/total for p in probs]
                next_word = random.choices(words, weights=probs, k=1)[0]
            else:
                break

        elif model_type == "trigram":

            if len(generated) < 2:
                break

            w1, w2 = generated[-2], generated[-1]
            next_word = sample_trigram_backoff(w1, w2)

        else:
            return None

        if next_word == "۔":
            break

        generated.append(next_word)

    return " ".join(generated)



headline_seeds = [
    "پاکستان میں",
    "حکومت نے",
    "وزیر اعظم",
    "ملک میں",
    "عوام کو"
]

for i in range(5):
    model = "bigram" if i % 2 == 0 else "trigram"
    headline = generate_headline(model, headline_seeds[i])
    print(f"{i+1}. {headline}")


1. پاکستان میں چھٹا فاصلہ پیجرز کھلونا بہٹ منصو عناصر بیھٹتے ممبئا سیہان
2. حکومت نے راونہ معیز کاوبار نیم لوکیشنز ریڈاٹ تر وسطا انشورڈ برتن
3. وزیر اعظم کھیل ڈیزیگنیشن غیرمقاما قمع دائرہ اظہار اجنبا ریاست فائنڈنگ مکنڈ
4. ملک میں آفرز خاوند داخل بہانہ مقررین جوفرنٹیئر اڑایا جہازکے قطرے بلکہ
5. عوام کو سزائے شاناں زیرانتطام بدر باپندا سموئے جع مائیکل قومیت مانا
